# Install Pytorch

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
print("platform, accelerator:", platform, accelerator)
!pip install -v -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision

# Imports

In [0]:
from __future__ import print_function
from PIL import Image
import os
import os.path
import errno
import numpy as np
import sys
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle

import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity

# Utilities

## Time Util

In [0]:
import datetime
import time

def get_time():
  ts = time.time()
  return str(datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

get_time()

## Google Drive Integration Utils

In [0]:
is_g_drive_setup = None
def setup_googleDrive():
  !pip install -U -q PyDrive
  !git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
  !mv /tmp/colab_util_repo/colab_util.py colab_util.py 
  !rm -r /tmp/colab_util_repo
  is_g_drive_setup = True

In [0]:
drive_handler = None
def setup_drive_handler():
  if not is_g_drive_setup:
    setup_googleDrive()
  drive_handler = GoogleDriveHandler()

def upload_to_google_drive(filename):
  if not drive_handler:
    setup_drive_handler()
  drive_handler.upload(filename, parent_path='test_folder')

def download_from_google_drive(filename, local_file_name):
  if not drive_handler:
    setup_drive_handler()
  drive_handler.download(filename, local_file_name)

def test_g_drive():
  !touch emptyFile2
  !echo "Hi" > emptyFile2
  upload_to_google_drive("emptyFile2")
  
# test_g_drive()

## CIFAR10 Dataset Utilities 

In [0]:
class CIFAR10(data.Dataset):
    """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    Args:
        root (string): Root directory of dataset where directory
            ``cifar-10-batches-py`` exists.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    base_folder = 'cifar-10-batches-py'
    url = "http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    filename = "cifar-10-python.tar.gz"
    tgz_md5 = 'c58f30108f718f92721af3b95e74349a'
    # validation examples will come from here
    train_list = [
        ['data_batch_1', 'c99cafc152244af753f735de768cd75f'],
        ['data_batch_2', 'd4bba439e000b95fd0a9bffe97cbabec'],
        ['data_batch_3', '54ebc095f3ab1f0389bbae665268c751'],
        ['data_batch_4', '634d18415352ddfa80567beed471001a'],
        ['data_batch_5', '482c414d41f54cd18b22e5b47cb7c3cb'],
    ]

    test_list = [
        ['test_batch', '40351d587109b95175f43aff81a1287e'],
    ]

    def __init__(self, root, split='train',
                 transform=None, target_transform=None,
                 download=False, val_samples=1000):
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.split = split # train, val, or test

        if download:
            self.download()

        if not self._check_integrity():
            raise RuntimeError('Dataset not found or corrupted.' +
                               ' You can use download=True to download it')

        # now load the picked numpy arrays
        if self.split in ['train', 'val']:
            self.train_data = []
            self.train_labels = []
            for fentry in self.train_list:
                f = fentry[0]
                file = os.path.join(self.root, self.base_folder, f)
                fo = open(file, 'rb')
                if sys.version_info[0] == 2:
                    entry = pickle.load(fo)
                else:
                    entry = pickle.load(fo, encoding='latin1')
                self.train_data.append(entry['data'])
                if 'labels' in entry:
                    self.train_labels += entry['labels']
                else:
                    self.train_labels += entry['fine_labels']
                fo.close()

            self.train_data = np.concatenate(self.train_data)
            self.train_data = self.train_data.reshape((50000, 3, 32, 32))
            self.train_data = self.train_data.transpose((0, 2, 3, 1))  # convert to HWC
            self.val_data = self.train_data[-val_samples:]
            self.val_labels = self.train_labels[-val_samples:]
            self.train_data = self.train_data[:-val_samples]
            self.train_labels = self.train_labels[:-val_samples]
        elif self.split == 'test':
            f = self.test_list[0][0]
            file = os.path.join(self.root, self.base_folder, f)
            fo = open(file, 'rb')
            if sys.version_info[0] == 2:
                entry = pickle.load(fo)
            else:
                entry = pickle.load(fo, encoding='latin1')
            self.test_data = entry['data']
            if 'labels' in entry:
                self.test_labels = entry['labels']
            else:
                self.test_labels = entry['fine_labels']
            fo.close()
            self.test_data = self.test_data.reshape((10000, 3, 32, 32))
            self.test_data = self.test_data.transpose((0, 2, 3, 1))  # convert to HWC
        else:
            raise Exception('Unkown split {}'.format(self.split))

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        if self.split == 'train':
            img, target = self.train_data[index], self.train_labels[index]
        elif self.split == 'val':
            img, target = self.val_data[index], self.val_labels[index]
        elif self.split == 'test':
            img, target = self.test_data[index], self.test_labels[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img)

        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        if self.split == 'train':
            return len(self.train_data)
        elif self.split == 'val':
            return len(self.val_data)
        elif self.split == 'test':
            return len(self.test_data)

    def _check_integrity(self):
        root = self.root
        for fentry in (self.train_list + self.test_list):
            filename, md5 = fentry[0], fentry[1]
            fpath = os.path.join(root, self.base_folder, filename)
            if not check_integrity(fpath, md5):
                return False
        return True

    def download(self):
        import tarfile

        if self._check_integrity():
            print('Files already downloaded and verified')
            return

        root = self.root
        download_url(self.url, root, self.filename, self.tgz_md5)

        # extract file
        cwd = os.getcwd()
        tar = tarfile.open(os.path.join(root, self.filename), "r:gz")
        os.chdir(root)
        tar.extractall()
        tar.close()
        os.chdir(cwd)



# Core Implementation

In [0]:
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from torch.autograd import Variable

## Train

In [0]:
def train(epoch, model, train_loader, is_cuda, criterion, optimizer, log_interval, val_loader):
    ret_strs = []
    model.train()
    for batch_idx, batch in enumerate(train_loader):
#         print("Epoch: ", epoch, "BatchIdx: ", batch_idx, batch[0].shape[0])
        images, targets = Variable(batch[0]), Variable(batch[1])
        if is_cuda:
            images, targets = images.cuda(), targets.cuda()
        loss = criterion(model(images), targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0: 
            val_loss, val_acc = evaluate(is_cuda=is_cuda, split='val', model=model, criterion=criterion, n_batches=4, loader=val_loader)
            train_loss = loss.data[0]
            examples_this_epoch = batch_idx * len(images)
            epoch_progress = 100. * batch_idx / len(train_loader)
            log_string = 'Train Epoch: {} [{}/{} ({:.0f}%)]\t Train Loss: {:.6f}\tVal Loss: {:.6f}\tVal Acc: {}'.format(
                epoch, examples_this_epoch, len(train_loader.dataset), epoch_progress, train_loss, val_loss, val_acc)
            print(log_string)
            ret_strs.append(log_string)

    return ret_strs

## Evaluate

In [0]:
def evaluate(is_cuda, split, model, loader, criterion, verbose=True, n_batches=None):
    """
    Compute loss on val or test data.
    """
    model.eval()
    loss = 0
    correct = 0
    n_examples = 0
    for batch_i, batch in enumerate(loader):
        data, target = batch
        if is_cuda:
            data, target = data.cuda(), target.cuda()
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
        output = model(data)
        loss += criterion(output, target, size_average=False).data[0]
        # predict the argmax of the log-probabilities
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        n_examples += pred.size(0)
        if n_batches and (batch_i >= n_batches):
            break

    loss /= n_examples
    acc = 100. * correct / n_examples
    if verbose:
        print('{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
            split, loss, correct, n_examples, acc))
    return loss, acc

## Plot Training curves

In [0]:
import matplotlib
# This is needed to save images 
# matplotlib.use('Agg')

import matplotlib.pyplot as plt
%matplotlib inline
import re

def parse_log_string(f):
    # Parse the train and val losses one line at a time.

    # regexes to find train and val losses on a line
    float_regex = r'[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?'
    train_loss_re = re.compile('.*Train Loss: ({})'.format(float_regex))
    val_loss_re = re.compile('.*Val Loss: ({})'.format(float_regex))
    val_acc_re = re.compile('.*Val Acc: ({})'.format(float_regex))
    
    # extract one loss for each logged iteration
    train_losses = []
    val_losses = []
    val_accs = []
        
    for line in f:
        train_match = train_loss_re.match(line)
        val_match = val_loss_re.match(line)
        val_acc_match = val_acc_re.match(line)
        if train_match:
            train_losses.append(float(train_match.group(1)))
        if val_match:
            val_losses.append(float(val_match.group(1)))
        if val_acc_match:
            val_accs.append(float(val_acc_match.group(1)))
    
    return train_losses, val_losses, val_accs
    
def plot_curves(train_losses, val_losses, val_accs, clf_name):
    fig = plt.figure()
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Val')
    plt.title(clf_name + 'Learning Curve')
    plt.ylabel('Loss')
    plt.legend()

    fig = plt.figure()
    plt.plot(val_accs, label='Val')
    plt.title(clf_name + ' Validation Accuracy During Training')
    plt.ylabel('Accuracy')
    plt.legend()
    
def tr_curves(log_str, clf_name="My Model"):
    train_losses, val_losses, val_accs = parse_log_string(log_str)
    plot_curves(train_losses, val_losses, val_accs, clf_name)

## Core Executor

- Loads the model
- Loads the dataset
- Defines the optimizer and loss function
- Trains the model for provided number of epochs

In [0]:
def main(lr, momentum, epochs, model_name, hidden_dim, kernel_size, weight_decay=0.0, batch_size=512, seed=1, 
               test_batch_size=1000, log_interval=10, cifar_10_dir="data", load_model_file=None):
  
  is_cuda = torch.cuda.is_available()
  kwargs = {'num_workers': 1, 'pin_memory': True} if is_cuda else {}

  torch.manual_seed(seed)

  n_classes = 10
  im_size = (3, 32, 32)

  cifar10_mean_color = [0.49131522, 0.48209435, 0.44646862]
  # std dev of color across training images
  cifar10_std_color = [0.01897398, 0.03039277, 0.03872553]
  
  transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(cifar10_mean_color, cifar10_std_color),
  ])
  
  # Model Loader
  if load_model_file:
    print("Loading model...")
    model = torch.load(load_model_file)
    print("Loaded model...")
  else:
    if model_name == 'softmax':
      model = Softmax(im_size, n_classes)
    elif model_name == 'twolayernn':
        model = TwoLayerNN(im_size, hidden_dim, n_classes)
    elif model_name == 'convnet':
        model = CNN(im_size, hidden_dim, kernel_size, n_classes)
    elif model_name == 'mymodel':
        model = MyModel(im_size, hidden_dim, kernel_size, n_classes)
    else:
        raise Exception('Unknown model {}'.format(args.model))
  
  # Datasets
  train_dataset = CIFAR10(cifar_10_dir, split='train', download=True, transform=transform)
  val_dataset = CIFAR10(cifar_10_dir, split='val', download=True, transform=transform)
  test_dataset = CIFAR10(cifar_10_dir, split='test', download=True, transform=transform)

  # DataLoaders
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

  if is_cuda:
      model.cuda()

  criterion = F.cross_entropy
# optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
  optimizer = optim.RMSprop(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
  scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 9, 12, 18], gamma=0.3)

  ret_strs = []
  for epoch in range(1, epochs + 1):
      scheduler.step()
      ret_strs = ret_strs + train(epoch, model, train_loader, is_cuda, criterion, optimizer, log_interval, val_loader)
      file_name = model_name + "_after5_" + get_time() + str(epoch) + '.pt'
      torch.save(model, file_name)

  evaluate(is_cuda=is_cuda, split='test', criterion=criterion, verbose=True, model=model, loader=test_loader)
  tr_curves(ret_strs)
  torch.save(model, model_name+get_time()+'_final.pt')


# Model Definition

## Softmax Classifier

In [0]:

class Softmax(nn.Module):
    def __init__(self, im_size, n_classes):
        super(Softmax, self).__init__()
        ch, h, w = im_size
        C = n_classes
        self.model = torch.nn.Sequential(
            nn.Linear(ch * h * w, C, bias=True),
            nn.ReLU(),
            nn.Softmax()
        )

    def forward(self, images):
        scores = None
        N, C, H, W = images.shape
        scores = self.model(images.reshape(N, C*H*W))
        return scores

## Custom ConvNet Classifier

**Model Description**

- Input 
- [ Conv[512 x 3 x 3]  - ReLU - BatchNorm - MaxPool ] 
- [ Conv[1024 x 3 x 3]  - ReLU - BatchNorm - MaxPool ] 
- [ Conv[1536 x 3 x 3]  - ReLU - BatchNorm - MaxPool ] 
- [ Conv[2048 x 3 x 3]  - ReLU - BatchNorm - MaxPool ] 
- [ FC[8192 x 4096 ] - ReLU -BatchNorm ] 
- [ FC[4096 x 2048 ] - ReLU -BatchNorm ]  
- [ FC[2048 x C ] - ReLU -BatchNorm ] 
- Softmax

In [0]:
class MyModel(nn.Module):
  def __init__(self, im_size, hidden_dim, kernel_size, n_classes):
      """
      Extra credit model

      Arguments:
          im_size (tuple): A tuple of ints with (channels, height, width)
          hidden_dim (int): Number of hidden activations to use
          kernel_size (int): Width and height of (square) convolution filters
          n_classes (int): Number of classes to score
      """
      super(MyModel, self).__init__()
      ch, h, w = im_size
      C = n_classes

      self.conv_block_1 = torch.nn.Sequential(
          nn.Conv2d(in_channels=ch, out_channels=512, kernel_size=(kernel_size, kernel_size), padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(512),
#           nn.MaxPool2d(kernel_size=(2, 2))
      )

      self.conv_block_2 = torch.nn.Sequential(
          nn.Conv2d(in_channels=512, out_channels=512 * 2, kernel_size=(kernel_size, kernel_size), padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(512 * 2),
          nn.MaxPool2d(kernel_size=(3, 3))
      )

      self.conv_block_3 = torch.nn.Sequential(
          nn.Conv2d(in_channels=512 * 2, out_channels=512 * 3, kernel_size=(kernel_size, kernel_size), padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(512 * 3),
#           nn.MaxPool2d(kernel_size=(2, 2))
      )

      self.conv_block_4 = torch.nn.Sequential(
          nn.Conv2d(in_channels=512 * 3, out_channels=512 * 4, kernel_size=(kernel_size, kernel_size), padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(512 * 4),
          nn.MaxPool2d(kernel_size=(3, 3))
      )

      self.conv_blocks = nn.Sequential(
          self.conv_block_1,
          self.conv_block_2,
          self.conv_block_3,
          self.conv_block_4
      )

      nn.init.xavier_normal_(self.conv_blocks[0][0].weight)
      nn.init.xavier_normal_(self.conv_blocks[1][0].weight)
      nn.init.xavier_normal_(self.conv_blocks[2][0].weight)
      nn.init.xavier_normal_(self.conv_blocks[3][0].weight)

      self.fcn_1 = torch.nn.Sequential(
          nn.Linear(512 * 4 * 3 * 3, 8192),
          nn.ReLU(),

          nn.Linear(8192, 4096),
          nn.ReLU(),
          nn.BatchNorm1d(4096),
          
          nn.Linear(4096, 2048),
          nn.ReLU(),
          
          nn.Linear(2048, C),
#           nn.ReLU(),
          nn.Softmax(1)
      )
      nn.init.xavier_normal_(self.fcn_1[0].weight)
      nn.init.xavier_normal_(self.fcn_1[2].weight)
      nn.init.xavier_normal_(self.fcn_1[5].weight)
      nn.init.xavier_normal_(self.fcn_1[7].weight)

  def forward(self, images):
      scores = None
      N = images.shape[0]
      scores = self.fcn_1(self.conv_blocks(images).reshape(N, -1))
      return scores


# Executor

In [0]:
import torch as torch
main(lr = 1e-6, 
     momentum = 0.9, 
     weight_decay = 1e-4, 
     batch_size = 64, 
     epochs = 20, 
     model_name = "mymodel", 
     hidden_dim = 50, 
     kernel_size = 3,
     seed = 1,            
     test_batch_size = 1000, 
     log_interval = 30, 
     cifar_10_dir = "data"
    )

In [0]:
# upload_to_google_drive("mymodel_after5_2018-10-02 21:37:1917.pt")

# Evaluation 

In [0]:
# !/opt/bin/nvidia-smi
# !ps -few


In [0]:
# NOTE: The scaffolding code for this part of the assignment
# is adapted from https://github.com/pytorch/examples.
from __future__ import print_function
import argparse
import os
import sys
import numpy as np
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import urllib
from torchvision import transforms
from torch.autograd import Variable

In [0]:
class ChallengeData(data.Dataset):
    """`CIFAR10 <https://www.cs.toronto.edu/~kriz/cifar.html>`_ Dataset.
    Args:
        root (string): Root directory of dataset where directory
            ``test_images.npy`` exists.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    url = "https://s3.amazonaws.com/cs7643-fall2018/test_images.npy"
    filename = "test_images.npy"

    def __init__(self, root,
                 transform=None, download=False):
        self.root = os.path.expanduser(root)
        self.transform = transform

        if download:
            self.download()

        # now load the picked numpy arrays
        file = os.path.join(self.root, self.filename)
        self.test_data = np.load(file)

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (image, target) where target is index of the target class.
        """
        img = self.test_data[index]

        # doing this so that it is consistent with all other datasets
        # to return a PIL Image
        img = Image.fromarray(img.astype('uint8'))

        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.test_data)

    def download(self):
        root = self.root
        if not os.path.exists(os.path.join(root, self.filename)):
            print("Downloading data...")
            urllib.request.urlretrieve(self.url, os.path.join(root, self.filename))
            print("Download complete")


In [0]:
def test_preds_csv(model="mymodel_after5_2018-10-02 21:37:1917.pt", test_dir="data", test_batch_size=256, no_cuda=False):

  # Load CIFAR10 using torch data paradigm
  kwargs = {'num_workers': 1, 'pin_memory': True}

  cifar10_mean_color = [0.49131522, 0.48209435, 0.44646862]
  # std dev of color across training images
  cifar10_std_color = [0.01897398, 0.03039277, 0.03872553]

  transform = transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize(cifar10_mean_color, cifar10_std_color),
              ])
  test_dataset = ChallengeData(test_dir, download=True, transform=transform)
  # Datasets
  test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, **kwargs)

  if os.path.exists(model):
      model = torch.load(model)
  else:
      print('Model path specified does not exst')
      sys.exit(1)

  # cross-entropy loss function
  criterion = F.cross_entropy
  if not no_cuda:
      model.cuda()


  def evaluate():
      '''
      Compute loss on test data.
      '''
      model.eval()
      loader = test_loader
      predictions = [] 
      for batch_i, batch in enumerate(loader):
          data = batch
          if not no_cuda:
              data= data.cuda()
          data = Variable(data, volatile=True)
          output = model(data)
          pred = output.data.max(1, keepdim=True)[1]
          predictions += pred.reshape(-1).tolist()
          print('Batch:{}'.format(batch_i))
      return predictions

  predictions = evaluate()

  with open('predictions.csv', 'w') as csv_file:
      csv_writer = csv.writer(csv_file, delimiter=',')
      csv_writer.writerow(['image_id', 'label'])
      for i, p in enumerate(predictions):
          csv_writer.writerow([i, int(p)])

In [0]:
# test_preds_csv()